In [1]:
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.general_recommender import LightGCN
from recbole.trainer import Trainer, PretrainTrainer
from recbole.utils import init_seed, init_logger
from recbole.utils import get_model, get_trainer
from recbole.utils.case_study import full_sort_topk, full_sort_scores

In [2]:
config = Config(model='LightGCN', dataset='Custom_ml-100k_2')

In [3]:
# init random seed
init_seed(config['seed'], config['reproducibility'])

In [4]:
# logger initialization
init_logger(config)
logger = getLogger()

In [5]:
# write config info into log
logger.info(config)

07 Feb 18:14    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2023
state = INFO
reproducibility = True
data_path = /home/RecBole/dataset/Custom_ml-100k_2
checkpoint_dir = /home/RecBole/saved/
show_progress = True
save_dataset = True
dataset_save_path = saved_dataset
save_dataloaders = True
dataloaders_save_path = saved_dataloaders
log_wandb = True

Training Hyper Parameters:
epochs = 100
train_batch_size = 4048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 1000
clip_grad_norm = None
weight_decay = 0.0001
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.675, 0.075, 0.25]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'MAP', 'Precision', 'GAUC', 'ItemCoverage', 'AveragePopularity', 'GiniIndex', 'ShannonEntropy', 'TailPercentag

In [6]:
# dataset creating and filtering
dataset = create_dataset(config)
logger.info(dataset)

07 Feb 18:14    INFO  Saving filtered dataset into [/home/RecBole/saved/Custom_ml-100k_2-dataset.pth]
07 Feb 18:14    INFO  Custom_ml-100k_2
The number of users: 587
Average actions of users: 85.32423208191126
The number of items: 1622
Average actions of items: 30.845157310302284
The number of inters: 50000
The sparsity of the dataset: 94.74852801240188%
Remain Fields: ['user_id', 'item_id', 'rating', 'timestamp']


In [7]:
# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

07 Feb 18:14    INFO  Saving split dataloaders into: [/home/RecBole/saved/Custom_ml-100k_2-for-LightGCN-dataloader.pth]
07 Feb 18:14    INFO  [Training]: train_batch_size = [4048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
07 Feb 18:14    INFO  [Evaluation]: eval_batch_size = [4048] eval_args: [{'split': {'RS': [0.675, 0.075, 0.25]}, 'group_by': 'user', 'order': 'RO', 'mode': 'full'}]


In [8]:
len(train_data)

9

In [9]:
train_data

In [10]:
len(valid_data)

292

In [11]:
len(test_data)

293

In [12]:
test_data

In [13]:
# model loading and initialization
model = LightGCN(config, train_data.dataset).to(config['device'])
logger.info(model)

07 Feb 18:14    INFO  LightGCN(
  (user_embedding): Embedding(587, 64)
  (item_embedding): Embedding(1622, 64)
  (mf_loss): BPRLoss()
  (reg_loss): EmbLoss()
)
Trainable parameters: 141376


In [14]:
trainer = Trainer(config, model)

07 Feb 18:14    ERROR  Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: d9249. Use `wandb login --relogin` to force relogin


07 Feb 18:14    WARNING  The parameters [weight_decay] and [reg_weight] are specified simultaneously, which may lead to double regularization.


In [15]:
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)

07 Feb 18:14    INFO  epoch 0 training [time: 0.27s, train loss: 5.4081]


interaction :  The batch_size of interaction: 2
    user_id, torch.Size([2]), cpu, torch.int64


scores :  tensor([[  -inf, 1.6738, 1.5823,  ..., 0.1810, 0.1725, 0.1835],
        [  -inf, 0.8600, 0.7271,  ..., 0.0814, 0.0825, 0.0814]],
       device='cuda:0')
scores size :  torch.Size([2, 1622])
positive_u :  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
positive_i:  tensor([224, 194, 663, 393, 425, 464, 820, 375, 152, 306])
result :  OrderedDict([('recall@20', 0.1823), ('mrr@20', 0.1451), ('ndcg@20', 0.1122), ('hit@20', 0.476), ('map@20', 0.0599), ('precision@20', 0.0342), ('gauc', 0.8231), ('itemcoverage@20', 0.0419), ('averagepopularity@20', 139.2655), ('giniindex@20', 0.9819), ('shannonentropy@20', 0.0529), ('tailpercentage@20', 0.0), ('F1', 0.05759501154734411)])


07 Feb 18:14    INFO  epoch 0 evaluating [time: 0.57s, valid_score: 0.145100]
07 Feb 18:14    INFO  valid result: 
recall@20 : 0.1823    mrr@20 : 0.1451    ndcg@20 : 0.1122    hit@20 : 0.476    map@20 : 0.0599    precision@20 : 0.0342    gauc : 0.8231    itemcoverage@20 : 0.0419    averagepopularity@20 : 139.2655    giniindex@20 : 0.9819    shannonentropy@20 : 0.0529    tailpercentage@20 : 0.0    F1 : 0.05759501154734411
07 Feb 18:14    INFO  Saving current: /home/RecBole/saved/LightGCN-Feb-07-2023_18-14-24.pth
07 Feb 18:14    INFO  epoch 1 training [time: 0.05s, train loss: 3.9128]


interaction :  The batch_size of interaction: 2
    user_id, torch.Size([2]), cpu, torch.int64


scores :  tensor([[  -inf, 4.6692, 4.5474,  ..., 0.5362, 0.4549, 0.5421],
        [  -inf, 2.3964, 2.0916,  ..., 0.2437, 0.2153, 0.2454]],
       device='cuda:0')
scores size :  torch.Size([2, 1622])
positive_u :  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
positive_i:  tensor([224, 194, 663, 393, 425, 464, 820, 375, 152, 306])
result :  OrderedDict([('recall@20', 0.1839), ('mrr@20', 0.1458), ('ndcg@20', 0.1126), ('hit@20', 0.476), ('map@20', 0.0592), ('precision@20', 0.0354), ('gauc', 0.824), ('itemcoverage@20', 0.0444), ('averagepopularity@20', 139.7841), ('giniindex@20', 0.9817), ('shannonentropy@20', 0.0502), ('tailpercentage@20', 0.0), ('F1', 0.05937127222982217)])


07 Feb 18:14    INFO  epoch 1 evaluating [time: 0.22s, valid_score: 0.145800]
07 Feb 18:14    INFO  valid result: 
recall@20 : 0.1839    mrr@20 : 0.1458    ndcg@20 : 0.1126    hit@20 : 0.476    map@20 : 0.0592    precision@20 : 0.0354    gauc : 0.824    itemcoverage@20 : 0.0444    averagepopularity@20 : 139.7841    giniindex@20 : 0.9817    shannonentropy@20 : 0.0502    tailpercentage@20 : 0.0    F1 : 0.05937127222982217
07 Feb 18:14    INFO  Saving current: /home/RecBole/saved/LightGCN-Feb-07-2023_18-14-24.pth
07 Feb 18:14    INFO  epoch 2 training [time: 0.06s, train loss: 3.7519]


...

07 Feb 18:14    INFO  epoch 97 evaluating [time: 0.20s, valid_score: 0.228300]
07 Feb 18:14    INFO  valid result: 
recall@20 : 0.2819    mrr@20 : 0.2283    ndcg@20 : 0.1756    hit@20 : 0.6336    map@20 : 0.091    precision@20 : 0.061    gauc : 0.8873    itemcoverage@20 : 0.2731    averagepopularity@20 : 102.2962    giniindex@20 : 0.9239    shannonentropy@20 : 0.0115    tailpercentage@20 : 0.0    F1 : 0.10029687955672208
07 Feb 18:14    INFO  epoch 98 training [time: 0.07s, train loss: 1.6507]


interaction :  The batch_size of interaction: 2
    user_id, torch.Size([2]), cpu, torch.int64


scores :  tensor([[   -inf,  1.4077,  4.0121,  ...,  0.6275, -0.1870,  0.5787],
        [   -inf,  2.9762,  1.6785,  ...,  0.2157,  0.2905,  0.1149]],
       device='cuda:0')
scores size :  torch.Size([2, 1622])
positive_u :  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
positive_i:  tensor([224, 194, 663, 393, 425, 464, 820, 375, 152, 306])
result :  OrderedDict([('recall@20', 0.2771), ('mrr@20', 0.2291), ('ndcg@20', 0.1744), ('hit@20', 0.6301), ('map@20', 0.0907), ('precision@20', 0.0604), ('gauc', 0.8877), ('itemcoverage@20', 0.2676), ('averagepopularity@20', 102.8227), ('giniindex@20', 0.9252), ('shannonentropy@20', 0.0117), ('tailpercentage@20', 0.0), ('F1', 0.09918127407407408)])


07 Feb 18:14    INFO  epoch 98 evaluating [time: 0.19s, valid_score: 0.229100]
07 Feb 18:14    INFO  valid result: 
recall@20 : 0.2771    mrr@20 : 0.2291    ndcg@20 : 0.1744    hit@20 : 0.6301    map@20 : 0.0907    precision@20 : 0.0604    gauc : 0.8877    itemcoverage@20 : 0.2676    averagepopularity@20 : 102.8227    giniindex@20 : 0.9252    shannonentropy@20 : 0.0117    tailpercentage@20 : 0.0    F1 : 0.09918127407407408
07 Feb 18:14    INFO  epoch 99 training [time: 0.09s, train loss: 1.6826]


interaction :  The batch_size of interaction: 2
    user_id, torch.Size([2]), cpu, torch.int64


scores :  tensor([[   -inf,  1.1590,  3.8777,  ...,  0.5989, -0.1728,  0.5715],
        [   -inf,  2.9154,  1.6522,  ...,  0.2071,  0.3007,  0.1068]],
       device='cuda:0')
scores size :  torch.Size([2, 1622])
positive_u :  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
positive_i:  tensor([224, 194, 663, 393, 425, 464, 820, 375, 152, 306])
result :  OrderedDict([('recall@20', 0.277), ('mrr@20', 0.2313), ('ndcg@20', 0.175), ('hit@20', 0.6284), ('map@20', 0.0912), ('precision@20', 0.0603), ('gauc', 0.8878), ('itemcoverage@20', 0.2719), ('averagepopularity@20', 102.1383), ('giniindex@20', 0.9236), ('shannonentropy@20', 0.0116), ('tailpercentage@20', 0.0), ('F1', 0.09904002371775868)])


07 Feb 18:14    INFO  epoch 99 evaluating [time: 0.20s, valid_score: 0.231300]
07 Feb 18:14    INFO  valid result: 
recall@20 : 0.277    mrr@20 : 0.2313    ndcg@20 : 0.175    hit@20 : 0.6284    map@20 : 0.0912    precision@20 : 0.0603    gauc : 0.8878    itemcoverage@20 : 0.2719    averagepopularity@20 : 102.1383    giniindex@20 : 0.9236    shannonentropy@20 : 0.0116    tailpercentage@20 : 0.0    F1 : 0.09904002371775868


In [16]:
best_valid_score

0.2318

In [17]:
best_valid_result

OrderedDict([('recall@20', 0.2773),
             ('mrr@20', 0.2318),
             ('ndcg@20', 0.1758),
             ('hit@20', 0.625),
             ('map@20', 0.0917),
             ('precision@20', 0.0609),
             ('gauc', 0.8873),
             ('itemcoverage@20', 0.2645),
             ('averagepopularity@20', 102.5961),
             ('giniindex@20', 0.9247),
             ('shannonentropy@20', 0.0119),
             ('tailpercentage@20', 0.0),
             ('F1', 0.09986735659373153)])

In [18]:
model.eval()

LightGCN(
  (user_embedding): Embedding(587, 64)
  (item_embedding): Embedding(1622, 64)
  (mf_loss): BPRLoss()
  (reg_loss): EmbLoss()
)

In [19]:
test_result = trainer.evaluate(test_data)

07 Feb 18:14    INFO  Loading model structure and parameters from /home/RecBole/saved/LightGCN-Feb-07-2023_18-14-24.pth


interaction :  The batch_size of interaction: 1
    user_id, torch.Size([1]), cpu, torch.int64


scores :  tensor([[  -inf, 3.1158, 1.7313,  ..., 0.2414, 0.2857, 0.1401]],
       device='cuda:0')
scores size :  torch.Size([1, 1622])
positive_u :  tensor([0, 0, 0, 0, 0, 0])
positive_i:  tensor([514, 408, 654, 178, 440, 505])
result :  OrderedDict([('recall@20', 0.3162), ('mrr@20', 0.5959), ('ndcg@20', 0.3751), ('hit@20', 0.9094), ('map@20', 0.2211), ('precision@20', 0.2479), ('gauc', 0.9085), ('itemcoverage@20', 0.2756), ('averagepopularity@20', 101.1001), ('giniindex@20', 0.9213), ('shannonentropy@20', 0.0115), ('tailpercentage@20', 0.0), ('F1', 0.2779151923417833)])


In [20]:
len(test_result)

13

wandb: Waiting for W&B process to finish... (success).


In [19]:
checkpoint_file = '/home/RecBole/saved/LightGCN-Feb-07-2023_12-43-52.pth'
trainer.resume_checkpoint(checkpoint_file)

07 Feb 12:44    INFO  Checkpoint loaded. Resume training from epoch 97


In [68]:
import numpy as np
import pandas as pd

rating_df = pd.read_csv('/home/RecBole/dataset/ml-100k/input/u.data', sep='\t', 
                        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
                        parse_dates=['timestamp'])

rating_df_sorted_by_values = rating_df.sort_values(by='timestamp', ascending=True)
test = rating_df_sorted_by_values.iloc[90000:100000]
test_sorted_by_index = test.sort_index(ascending=True)
uid = test_sorted_by_index['user_id']
iid = test_sorted_by_index['movie_id']

In [21]:
uid.nunique()

166

In [70]:
iid.nunique()

1343

In [22]:
uid

1        186
7        253
19       234
21       167
26        38
        ... 
99942    363
99953    655
99987    660
99988    421
99991    676
Name: user_id, Length: 10000, dtype: int64

In [71]:
iid

1         302
7         465
19       1184
21        486
26         95
         ... 
99942     181
99953     913
99987     229
99988     498
99991     538
Name: movie_id, Length: 10000, dtype: int64

In [23]:
uid.unique()

array([186, 253, 234, 167,  38, 223, 189,  11,  90,  94, 269, 121, 279,
        26,  56, 125,  91, 229, 214,  33, 112, 215, 191,  61, 197, 134,
       111, 280, 146,  58,  14, 312,   7, 190, 179,   4, 252, 200, 204,
       247, 266,  54, 325,  85, 195, 183, 116,  13, 159,  27, 329, 119,
       334,  39, 335, 178, 102, 317, 328, 363, 356,  43, 233, 381, 332,
       299, 276, 353,  92, 378, 416, 411, 423, 379, 421, 428, 174, 440,
       294, 425, 448, 444, 472, 358, 407, 455, 232, 145, 489, 503, 475,
       532, 517, 219, 514, 533, 548, 551, 500, 453, 587, 603, 450, 611,
       483, 561, 499, 530, 624, 643, 625, 636, 645, 650, 653, 655, 649,
       656, 659, 669, 676, 683, 701, 704, 708, 207, 714, 729, 699, 738,
       741, 753, 767, 776, 782, 642, 784, 660, 796, 661, 807, 818, 825,
       829, 830, 828, 827, 816, 778, 851, 856, 810, 873, 880, 883, 891,
       911, 918, 922, 909, 751, 860, 903, 479, 790, 838])

In [72]:
iid.unique()

array([ 302,  465, 1184, ...,  833, 1140, 1610])

In [24]:
uid.value_counts()

234    428
796    358
551    334
334    333
269    323
      ... 
358      1
661      1
294      1
14       1
838      1
Name: user_id, Length: 166, dtype: int64

In [25]:
uid

1        186
7        253
19       234
21       167
26        38
        ... 
99942    363
99953    655
99987    660
99988    421
99991    676
Name: user_id, Length: 10000, dtype: int64

In [67]:
len(uid.unique())

166

In [54]:
uid_series = dataset.token2id(dataset.uid_field, ['1', '186'])
# uid_series = dataset.token2id(dataset.uid_field, ['7', '253'])

In [27]:
uid_series

array([  9, 408])

In [78]:
score = full_sort_scores(uid_series, model, test_data, device=config['device'])
print(score)  # score of all items

tensor([[       -inf,  3.8014e+00, -5.9394e-02,  ..., -4.5321e-01,
          3.3351e-01, -8.1701e-03],
        [       -inf,  5.2378e+00,  2.4305e-01,  ...,  3.8994e-01,
          4.5201e-03, -1.1946e-02]], device='cuda:0')


In [59]:
result = score[0, dataset.token2id(dataset.iid_field, ['40'])]
# score of item ['283', '113'] for user '196'.

In [73]:
dataset.token2id(dataset.iid_field, ['302'])

array([149])

In [96]:
score[1, 149]

tensor(5.8526, device='cuda:0')

In [75]:
result

tensor([2.5247], device='cuda:0')

In [76]:
score[1]

tensor([   -inf,  3.7294,  3.3661,  ...,  0.7369, -0.0687, -0.0322],
       device='cuda:0')

In [82]:
score_2 = full_sort_topk(uid_series, model, test_data, k=15, device=config['device'])
score_2  # score of all items

torch.return_types.topk(
values=tensor([[4.0923, 3.9093, 3.8014, 3.7448, 3.6433, 3.4447, 3.4372, 3.2954, 3.1395,
         2.9601, 2.9410, 2.8891, 2.8666, 2.8603, 2.8497],
        [7.5915, 7.0294, 6.2821, 6.2224, 6.1681, 6.1402, 6.0532, 5.8526, 5.7952,
         5.5251, 5.4144, 5.3703, 5.3340, 5.3029, 5.2378]], device='cuda:0'),
indices=tensor([[ 40, 375,   1, 319, 346, 226, 149, 265, 256, 243, 492, 527, 382, 306,
         489],
        [319, 275, 375,  46, 154,  26, 382, 149, 111,  99,  51, 346, 306,  28,
           1]], device='cuda:0'))

In [88]:
score_2[0][0]

tensor([4.0923, 3.9093, 3.8014, 3.7448, 3.6433, 3.4447, 3.4372, 3.2954, 3.1395,
        2.9601, 2.9410, 2.8891, 2.8666, 2.8603, 2.8497], device='cuda:0')

In [87]:
score_2[0][1]

tensor([7.5915, 7.0294, 6.2821, 6.2224, 6.1681, 6.1402, 6.0532, 5.8526, 5.7952,
        5.5251, 5.4144, 5.3703, 5.3340, 5.3029, 5.2378], device='cuda:0')

In [90]:
score_2[1][0]

tensor([ 40, 375,   1, 319, 346, 226, 149, 265, 256, 243, 492, 527, 382, 306,
        489], device='cuda:0')

In [91]:
score_2[1][1]

tensor([319, 275, 375,  46, 154,  26, 382, 149, 111,  99,  51, 346, 306,  28,
          1], device='cuda:0')

In [101]:
from recbole.evaluator.base_metric import TopkMetric

pos_index, _ = used_info(test_data)

NameError: name 'used_info' is not defined